In [ ]:
# default_exp io

# I2T2.io

> I2T2.io loads and saves common file formats used in medical imaging.

In [ ]:
import gdcm

In [ ]:
#test
import gdcm

In [ ]:
#test
try:
    import gdcm
except:
    print('gdcm is needed for compressed DICOM loading')

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
#test
assert say_hello("Jeremy")=="Hello Jeremy!"